# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.0752,-159.3190,24.99,91,100,8.75,US,1675482840
1,1,ushuaia,-54.8000,-68.3000,8.81,71,20,9.77,AR,1675482812
2,2,shima,24.4478,117.8111,14.01,73,100,1.32,CN,1675482286
3,3,pong,19.1493,100.2752,32.33,45,48,1.25,TH,1675482849
4,4,tuktoyaktuk,69.4541,-133.0374,-42.00,66,75,0.00,CA,1675482849


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = "Humidity",
    color = "City"
)

# Display the map 
map_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df[(city_data_df['Max Temp']>21.1) & (city_data_df['Max Temp']<23.9) & (city_data_df['Humidity']<80) & (city_data_df['Cloudiness']<50)]

# Drop any rows with null values
vacation_df = vacation_df.dropna()

# Display sample data
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,mangrol,21.1167,70.1167,22.56,33,1,3.96,IN,1675482851
137,137,port augusta,-32.5000,137.7667,22.62,44,11,6.75,AU,1675482882
194,194,kaitangata,-46.2817,169.8464,22.87,64,23,3.46,NZ,1675482849
198,198,itinga,-16.6131,-41.7653,23.54,72,43,1.16,BR,1675482899
212,212,bhainsdehi,21.6467,77.6325,21.75,16,11,0.58,IN,1675482956
232,232,disa,24.2500,72.1667,22.26,26,0,1.80,IN,1675482998
262,262,youkounkoun,12.5333,-13.1333,22.52,19,6,0.64,GN,1675483006
378,378,tura,25.5198,90.2201,23.22,37,2,1.15,IN,1675483070
421,421,presidencia roque saenz pena,-26.7852,-60.4388,22.77,56,47,2.54,AR,1675483082
471,471,rawson,-43.3002,-65.1023,23.22,45,0,5.53,AR,1675482803


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,mangrol,IN,21.1167,70.1167,33,
137,port augusta,AU,-32.5000,137.7667,44,
194,kaitangata,NZ,-46.2817,169.8464,64,
198,itinga,BR,-16.6131,-41.7653,72,
212,bhainsdehi,IN,21.6467,77.6325,16,
232,disa,IN,24.2500,72.1667,26,
262,youkounkoun,GN,12.5333,-13.1333,19,
378,tura,IN,25.5198,90.2201,37,
421,presidencia roque saenz pena,AR,-26.7852,-60.4388,56,
471,rawson,AR,-43.3002,-65.1023,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.at[index,'Lat']
    longitude = hotel_df.at[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mangrol - nearest hotel: No hotel found
port augusta - nearest hotel: Hotel Commonwealth
kaitangata - nearest hotel: Hotel South Otago
itinga - nearest hotel: No hotel found
bhainsdehi - nearest hotel: No hotel found
disa - nearest hotel: No hotel found
youkounkoun - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
presidencia roque saenz pena - nearest hotel: Hotel Presidente
rawson - nearest hotel: Hotel Deportivo
uruguaiana - nearest hotel: Casa Branca


,City,Country,Lat,Lng,Humidity,Hotel Name
9,mangrol,IN,21.1167,70.1167,33,No hotel found
137,port augusta,AU,-32.5000,137.7667,44,Hotel Commonwealth
194,kaitangata,NZ,-46.2817,169.8464,64,Hotel South Otago
198,itinga,BR,-16.6131,-41.7653,72,No hotel found
212,bhainsdehi,IN,21.6467,77.6325,16,No hotel found
232,disa,IN,24.2500,72.1667,26,No hotel found
262,youkounkoun,GN,12.5333,-13.1333,19,No hotel found
378,tura,IN,25.5198,90.2201,37,No hotel found
421,presidencia roque saenz pena,AR,-26.7852,-60.4388,56,Hotel Presidente
471,rawson,AR,-43.3002,-65.1023,45,Hotel Deportivo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = "Humidity",
    color = "City",
    hover_cols = ['City','Lng','Lat','Humidity','Hotel Name'],
    frame_width = 800,
    frame_height = 600
)

# Display the map 
map_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)